In [156]:
import os 
import pandas as pd

In [2]:
# Define directories that may be needed
home_dir = os.getcwd()
in_dir = '../../'

In [4]:
# Dataimport
# Note: the file import is currently custom and needs adjustment with new data
df = pd.read_csv(in_dir + 'BOL/exp/19350701_20250122_BOL_COMPLETE.csv', 
                 parse_dates = True, 
                 low_memory=False,
                 )

In [5]:
"""
SITES uses different names for the parameter headers, which need to be adjusted before uploading to the portal. 
This dict is meant to 'translate' this differences. 
"""

col_dict_SITES = {
    'Water depth [m]' : 'D_Water',
    'Water temperature [°C]' : 'TW',
    'EC [µS/cm]' : 'EC',
    'SCOND [µS/cm]' : 'SCOND',
    'pH' : 'pH', 
    'ORP [mV]' : 'ORP',
    'Turbidity [FNU]' : 'TURBF',
    'Chlorophyll [µg/L]' : 'CHLF',
    'O2SAT [%]' : 'O2SAT',
    'O2 [mg/L]' : 'O2',
    'fDOM [QSU]' : 'FDOM'
}

# List profile locations for SITES  
sites_profil = ['Bol NW', 'Bol NE', 'Bol E', 'Bol S', 'Bol W']

# Set last DN uploaded on SITES DataPortal
""" 
The DN of each location needs to be adjusted to allow consistency with the SITES dataportal. Currently the DN is not consistent due to previous handcleaning of the data. 
The following dictionary holds the information about the last DN uploaded for each location. 
"""

DN_portal = {'Bol NW' : 14, 
             'Bol NE' : 16, 
             'Bol E' : 16, 
             'Bol S' : 15, 
             'Bol W': 17
             }

# List of needed columns 
cols_of_interest = ['Time',
                    'Date',
                    'Site',
                    'DN',
                    'Water depth [m]',
                    'Water temperature [°C]',
                    'EC [µS/cm]',
                    'SCOND [µS/cm]',
                    'pH', 
                    'ORP [mV]',
                    'Turbidity [FNU]',
                    'Chlorophyll [µg/L]',
                    'O2SAT [%]',
                    'O2 [mg/L]',
                    'fDOM [QSU]'
                    ]

# Define sorting of columns to be in line with the header file found in 'SITES_headers'
sorting = ['TIMESTAMP', 'DN', 'D_Water', 
           'TW', 'EC', 'SCOND', 'pH',
           'ORP', 'TURBF', 'CHLF', 'O2SAT', 
           'O2', 'FDOM'
           ]

In [7]:
# select data
tmp = df[(df['Site'].isin(sites_profil)) & (df['Datasource']=='EXO_KOR') & (df['Date']>'2023-10-05')]
tmp = tmp[cols_of_interest]

# export data per station 
for site in sites_profil:
    # Select data from input dataset
    out = tmp[tmp['Site']==site]
    
    # Concatenate date and time into single column 
    out['TIMESTAMP'] = pd.to_datetime(tmp['Date'] + ' ' + tmp['Time'])
    out.drop(['Date', 'Time'], axis=1, inplace=True)
    
    # Rename columns to fit SITES
    out = out.rename(columns=col_dict_SITES)

    # Sort columns
    out = out.loc[:, sorting]

    # adjust DN
    out['DN'] = out['DN'] - (out['DN'].iloc[0] - 1)
    out['DN'] = (out['DN'] + DN_portal[site]).astype(int)

    # read header
    header = pd.read_csv(f"SITES_headers/head-PROF_{site}.csv",
                          sep=';', header=None,
                          skipfooter=1,
                          )
    
    ## adjust time period in header file
    start_date = out['TIMESTAMP'].iloc[0].date()
    end_date = out['TIMESTAMP'].iloc[-1].date()
    TimePeriod = f"{start_date} - {end_date}"
    header.iloc[2, 0] = header.iloc[2, 0].replace('yyyy-mm-dd - yyyy-mm-dd', TimePeriod)

    # remove seconds in timestamp
    out['TIMESTAMP'] = out['TIMESTAMP'].dt.strftime('%Y-%m-%d %H:%M')

    # temporary store data 
    out.to_csv('tmp/data.csv', index=None)
    header.to_csv('tmp/header.csv', index=None, header=None)

    # combine header and data
    ## Reading header from tmp
    with open('tmp/header.csv') as fp:
        data = fp.read().replace('"',"")

    ## Reading data from tmp
    with open('tmp/data.csv') as fp:
        data2 = fp.read()

    ## merging header and data
    data += data2

    # write output file
    start_date_iso = start_date.isoformat().replace('-','')
    end_date_iso = end_date.isoformat().replace('-','')
    with open (f"../exp/SITES_SONDE-PROF_BOL_BS-{site[4:]}_{start_date_iso}-{end_date_iso}_L2_irregular.csv", 'w') as fp:
        fp.write(data)

    # delete tmp output files
    os.remove('tmp/header.csv')
    os.remove('tmp/data.csv')
    